## 데이터 셋

In [ ]:
from __future__ import print_function
import torch.utils.data as data
from PIL import Image
import os
import os.path
import errno
import torch
import codecs


class fashion(data.Dataset):
    """`MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.
    Args:
        root (string): Root directory of dataset where ``processed/training.pt``
            and  ``processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    urls = [
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz',
    ]
    raw_folder = 'raw'
    processed_folder = 'processed'
    training_file = 'training.pt'
    test_file = 'test.pt'

    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +
                               ' You can use download=True to download it')

        if self.train:
            self.train_data, self.train_labels = torch.load(
                os.path.join(root, self.processed_folder, self.training_file))
        else:
            self.test_data, self.test_labels = torch.load(os.path.join(root, self.processed_folder, self.test_file))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_exists(self):
        return os.path.exists(os.path.join(self.root, self.processed_folder, self.training_file)) and \
            os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file))

    def download(self):
        """Download the MNIST data if it doesn't exist in processed_folder already."""
        from six.moves import urllib
        import gzip

        if self._check_exists():
            return

        # download files
        try:
            os.makedirs(os.path.join(self.root, self.raw_folder))
            os.makedirs(os.path.join(self.root, self.processed_folder))
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise

        for url in self.urls:
            print('Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[2]
            file_path = os.path.join(self.root, self.raw_folder, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())
            with open(file_path.replace('.gz', ''), 'wb') as out_f, \
                    gzip.GzipFile(file_path) as zip_f:
                out_f.write(zip_f.read())
            os.unlink(file_path)

        # process and save as torch files
        print('Processing...')

        training_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 'train-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 'train-labels-idx1-ubyte'))
        )
        test_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 't10k-labels-idx1-ubyte'))
        )
        with open(os.path.join(self.root, self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.root, self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')


def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def parse_byte(b):
    if isinstance(b, str):
        return ord(b)
    return b


def read_label_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2049
        length = get_int(data[4:8])
        labels = [parse_byte(b) for b in data[8:]]
        assert len(labels) == length
        return torch.LongTensor(labels)


def read_image_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2051
        length = get_int(data[4:8])
        num_rows = get_int(data[8:12])
        num_cols = get_int(data[12:16])
        images = []
        idx = 16
        for l in range(length):
            img = []
            images.append(img)
            for r in range(num_rows):
                row = []
                img.append(row)
                for c in range(num_cols):
                    row.append(parse_byte(data[idx]))
                    idx += 1
        assert len(images) == length
        return torch.ByteTensor(images).view(-1, 28, 28)

## Wandb 설정

In [ ]:
!pip install wandb
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import torch.nn.functional as F

import wandb
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
hyperparameter_defaults = dict(
    dropout = 0.5,
    channels_one = 16,
    channels_two = 32,
    batch_size = 128,
    learning_rate = 0.001,
    epochs = 2,
    )

wandb.init(config=hyperparameter_defaults, project="pytorch-cnn-fashion")
config = wandb.config

wandb: Currently logged in as: vinnyshin. Use `wandb login --relogin` to force relogin


## Sweep 초기화


In [ ]:
sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "metric" : {
      "goal" : "minimize",
      "name" : "loss"
  },
  "parameters" : {
    "dropout" : {
      "values" : [0.1, 0.2, 0.4, 0.5, 0.7]
    },
    "channels_one" : {
        "values" : [10, 12, 14, 16, 18, 20]
    },
    "channels_two" : {
        "values" : [24, 28, 32, 36, 40, 44]
    },
    "epochs" : {
      "values" : [10, 20, 50]
    },
    "learning_rate" : {
      "min": 0.0001,
      "max": 0.1
    }
  },
  "early_terminate": {
      "type": "hyperband",
      "s" : 2,
      "eta" : 3,
      "max_iter" : 27,
  }
}

sweep_id = wandb.sweep(sweep_config, project="pytorch-cnn-fashion")

Create sweep with ID: gdpr84rb
Sweep URL: https://wandb.ai/vinnyshin/pytorch-cnn-fashion/sweeps/gdpr84rb


## 모델 선언 및 학습

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=config.channels_one, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=config.channels_one, out_channels=config.channels_two, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        self.dropout = nn.Dropout(p=config.dropout)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(config.channels_two*4*4, 10)

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        # Linear function (readout)
        out = self.fc1(out)

        return out

In [ ]:
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                      std=[x/255.0 for x in [63.0, 62.1, 66.7]])

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = fashion(root='./data',
                            train=True,
                            transform=transform,
                            download=True
                            )

test_dataset = fashion(root='./data',
                            train=False,
                            transform=transform,
                            )

label_names = [
    "T-shirt or top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Boot"]

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=config.batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=config.batch_size,
                                          shuffle=False)


In [ ]:
model = CNNModel()
wandb.watch(model)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

In [ ]:
def train():
  wandb.init(config=hyperparameter_defaults)
  iter = 0
  for epoch in range(config.epochs):
      for i, (images, labels) in enumerate(train_loader):

          images = Variable(images)
          labels = Variable(labels)

          # Clear gradients w.r.t. parameters
          optimizer.zero_grad()

          # Forward pass to get output/logits
          outputs = model(images)

          # Calculate Loss: softmax --> cross entropy loss
          loss = criterion(outputs, labels)

          # Getting gradients w.r.t. parameters
          loss.backward()

          # Updating parameters
          optimizer.step()

          iter += 1

          if iter % 100 == 0:
              # Calculate Accuracy
              correct = 0.0
              correct_arr = [0.0] * 10
              total = 0.0
              total_arr = [0.0] * 10

              # Iterate through test dataset
              for images, labels in test_loader:
                  images = Variable(images)

                  # Forward pass only to get logits/output
                  outputs = model(images)

                  # Get predictions from the maximum value
                  _, predicted = torch.max(outputs.data, 1)

                  # Total number of labels
                  total += labels.size(0)
                  correct += (predicted == labels).sum()

                  for label in range(10):
                      correct_arr[label] += (((predicted == labels) & (labels==label)).sum())
                      total_arr[label] += (labels == label).sum()

              accuracy = correct / total

              metrics = {'accuracy': accuracy, 'loss': loss}
              for label in range(10):
                  metrics['Accuracy ' + label_names[label]] = correct_arr[label] / total_arr[label]

              wandb.log(metrics)

              # Print Loss
              print('Iteration: {0} Loss: {1:.2f} Accuracy: {2:.2f}'.format(iter, loss, accuracy))

  # torch.save(model.state_dict(), os.path.join(wandb.run.dir, "model.pt"))


In [ ]:
count = 30
wandb.agent(sweep_id, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 33ty1yi5 with config:
wandb: 	channels_one: 18
wandb: 	channels_two: 24
wandb: 	dropout: 0.5
wandb: 	epochs: 50
wandb: 	learning_rate: 0.04806176589570687


Iteration: 100 Loss: 0.67 Accuracy: 0.73
Iteration: 200 Loss: 0.65 Accuracy: 0.76
Iteration: 300 Loss: 0.48 Accuracy: 0.80
Iteration: 400 Loss: 0.44 Accuracy: 0.81
Iteration: 500 Loss: 0.51 Accuracy: 0.81
Iteration: 600 Loss: 0.40 Accuracy: 0.82
Iteration: 700 Loss: 0.49 Accuracy: 0.83
Iteration: 800 Loss: 0.31 Accuracy: 0.83
Iteration: 900 Loss: 0.39 Accuracy: 0.83


Accuracy Bag,▁▂▇▃▆▇█▇▇
Accuracy Boot,▆▁▃▇▆███▄
Accuracy Coat,▁▁▅▆▄█▆▄▅
Accuracy Dress,▄▁█▆▆▇██▇
Accuracy Pullover,▄▁▅█▅▆█▆█
Accuracy Sandal,▁▁▆▄██▇▆█
Accuracy Shirt,▁█▅▂▇▅▅█▇
Accuracy Sneaker,▁▆▆▆▆▆▆▇█
Accuracy T-shirt or top,▂▁▄▆▇▄▅█▆
Accuracy Trouser,▁▇▇▇█████
accuracy,▁▃▆▆▇▇▇██


wandb: Agent Starting Run: rfol1hfc with config:
wandb: 	channels_one: 18
wandb: 	channels_two: 32
wandb: 	dropout: 0.1
wandb: 	epochs: 50
wandb: 	learning_rate: 0.06490480056848175


Iteration: 100 Loss: 0.29 Accuracy: 0.83
Iteration: 200 Loss: 0.52 Accuracy: 0.84
Iteration: 300 Loss: 0.33 Accuracy: 0.85
Iteration: 400 Loss: 0.34 Accuracy: 0.85
Iteration: 500 Loss: 0.46 Accuracy: 0.85
Iteration: 600 Loss: 0.35 Accuracy: 0.84
Iteration: 700 Loss: 0.33 Accuracy: 0.85
Iteration: 800 Loss: 0.40 Accuracy: 0.85
Iteration: 900 Loss: 0.38 Accuracy: 0.85


...

Iteration: 100 Loss: 0.18 Accuracy: 0.89
Iteration: 200 Loss: 0.24 Accuracy: 0.89
Iteration: 300 Loss: 0.24 Accuracy: 0.89
Iteration: 400 Loss: 0.26 Accuracy: 0.89
Iteration: 500 Loss: 0.27 Accuracy: 0.89
Iteration: 600 Loss: 0.34 Accuracy: 0.88
Iteration: 700 Loss: 0.29 Accuracy: 0.88
Iteration: 800 Loss: 0.24 Accuracy: 0.88
Iteration: 900 Loss: 0.23 Accuracy: 0.88


Accuracy Bag,█▃▆▇▇▁▅▅▃
Accuracy Boot,▅▁▃▅▇▁▇█▃
Accuracy Coat,█▇▆█▆▇▁██
Accuracy Dress,▁▃▅█▃▂▅█▄
Accuracy Pullover,▄▇█▂█▄▆▁▃
Accuracy Sandal,▅▃▄▁▆▄█▆▇
Accuracy Shirt,▇▃▄█▆▁██▄
Accuracy Sneaker,▅▆▆▆▄█▁▂▅
Accuracy T-shirt or top,▂█▄▁▁▇▃▁▆
Accuracy Trouser,█▆▅▅▅█▇▁▅
accuracy,▆▇█▅▇▁▃▂▃


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4rth06by with config:
wandb: 	channels_one: 12
wandb: 	channels_two: 36
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	learning_rate: 0.05325372355251654


Iteration: 100 Loss: 0.18 Accuracy: 0.88
Iteration: 200 Loss: 0.18 Accuracy: 0.88
Iteration: 300 Loss: 0.19 Accuracy: 0.88
Iteration: 400 Loss: 0.23 Accuracy: 0.88
Iteration: 500 Loss: 0.26 Accuracy: 0.89
Iteration: 600 Loss: 0.23 Accuracy: 0.89
Iteration: 700 Loss: 0.20 Accuracy: 0.88
Iteration: 800 Loss: 0.23 Accuracy: 0.88
Iteration: 900 Loss: 0.26 Accuracy: 0.89


Accuracy Bag,▅▅▁▃▂▂█▅▅
Accuracy Boot,▁▃█▆▅▅▄▄▅
Accuracy Coat,█▁▅▂▄▃▃█▇
Accuracy Dress,▄▅▆█▅▅▇▁▅
Accuracy Pullover,▄▇▇██▆▇▁▆
Accuracy Sandal,▆▄█▅▆▄▆▅▁
Accuracy Shirt,▃▇▄▁▃█▁█▂
Accuracy Sneaker,▆█▁▄▇▇▆▄█
Accuracy T-shirt or top,▃▃▁▅▆▇█▅▆
Accuracy Trouser,▅▇▆▁▆▆▂█▇
accuracy,▆▃▄▁▆█▄▆█
